In [1]:
import pandas as pd

# 1.0 Limpieza base de datos ingresos

In [2]:
ingreso = pd.read_csv(r'conjunto_de_datos_ingresos_enigh_2018_ns.csv')

1. Seleccionamos hogar principal

In [4]:
hogar_principal = ingreso['foliohog'] ==1

ingresos_hogarprincipal =ingreso[hogar_principal]

1.1 limpiamos los espacios vacíos

In [5]:

df_updated = ingresos_hogarprincipal.replace(to_replace ='\s\s*', value = '0', regex = True) 

1.2 cambiamos ingresos a número

In [6]:
df_updated['ing_2'] = df_updated['ing_2'].astype(int)
df_updated['ing_3'] = df_updated['ing_3'].astype(int)
df_updated['ing_4'] = df_updated['ing_4'].astype(int)
df_updated['ing_5'] = df_updated['ing_5'].astype(int)
df_updated['ing_6'] = df_updated['ing_6'].astype(int)

1.3 sumo todos los ingresos del hogar: quiero el ingreso total de la vivienda

In [7]:
ingreso_todos=df_updated.groupby(
   ['folioviv']
).agg(
    {
         'ing_1':sum,   
         'ing_2':sum,
         'ing_3':sum,
        'ing_4':sum,
        'ing_5':sum,
        'ing_6':sum 
    }
)

In [8]:
ingreso_todos.head()

,ing_1,ing_2,ing_3,ing_4,ing_5,ing_6
folioviv,,,,,,
100013601,21100,22200,21100,22200,21100,22200
100013602,6248,5333,4913,4513,7913,4513
100013603,59000,59500,59500,59500,59500,59500
100013604,11500,10000,10000,10000,10000,10000
100013606,14700,14600,14700,14600,14700,14600


In [9]:
ingreso_todos.dtypes

ing_1    int64
ing_2    int64
ing_3    int64
ing_4    int64
ing_5    int64
ing_6    int64
dtype: object

1.4 queremos el ingreso promedio de los 6 meses

In [10]:
col = ingreso_todos.loc[: , "ing_1":"ing_6"]
ingreso_todos['mean_income'] = col.mean(axis=1)

1.5 con el aggregate me indexo la columna folioviv la quiero como columna

In [11]:
ingreso_todos=ingreso_todos.reset_index()

In [12]:
ingreso_todos.head()

,folioviv,ing_1,ing_2,ing_3,ing_4,ing_5,ing_6,mean_income
0,100013601,21100,22200,21100,22200,21100,22200,21650.000000
1,100013602,6248,5333,4913,4513,7913,4513,5572.166667
2,100013603,59000,59500,59500,59500,59500,59500,59416.666667
3,100013604,11500,10000,10000,10000,10000,10000,10250.000000
4,100013606,14700,14600,14700,14600,14700,14600,14650.000000


# 2.0 Limpieza base de datos gastos 

In [13]:
gastos_hogar = pd.read_csv(r'conjunto_de_datos_gastoshogar_enigh_2018_ns.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2. Seleccionamos hogar principal

In [14]:
hogar_principal = gastos_hogar['foliohog'] ==1

gastos_hogarprincipal =gastos_hogar[hogar_principal]

2.1 sólo quiero el gasto mensual conservo sólo las columnas que necesito

In [15]:
gastos_hogarprincipal_onlygastos = gastos_hogarprincipal[['folioviv', 'clave', 'gasto']]

2.2 verifico que no haya espacios en blanco y tipos de variables

In [21]:
df_updated = gastos_hogarprincipal.replace(to_replace ='\s\s*', value = '0', regex = True) 

In [23]:
df_updated['folioviv'] = df_updated['folioviv'].astype(str)
df_updated['clave'] = df_updated['clave'].astype(str)
df_updated['gasto'] = df_updated['gasto'].astype(int)

In [27]:
df_updated2 = df_updated[['folioviv', 'clave', 'gasto']]

In [30]:
df_updated2.to_csv(r'gastos_hogarprincipal_onlygastoos.csv')

In [31]:

gastos_hogarprincipal_onlygastos = pd.read_csv(r'gastos_hogarprincipal_onlygastoos.csv', dtype={'folioviv':'str', 'clave':'str','gasto':'int32'})

In [32]:
gastos_hogarprincipal_onlygastos.dtypes

Unnamed: 0     int64
folioviv      object
clave         object
gasto          int32
dtype: object

In [ ]:
# por clave de gasto no voy a saber de que gasto hablo necesito unir el catalogo de gastos

2.3 uniendo el catalogo de gastos

In [33]:
#ojo con el encoding o no voy a tener bien los acentos
catalogo_gastos=pd.read_csv(r'/content/gastos.csv', encoding='latin-1')

In [34]:
#tengo el mismo numero de hogares? cuales quiero conservar?
gastos_hogarprincipal = pd.merge(gastos_hogarprincipal_onlygastos, catalogo_gastos, left_on='clave', right_on='gastos', how='left')

In [36]:
del(gastos_hogarprincipal['Unnamed: 0'])

In [37]:
gastos_hogarprincipal.head()

,folioviv,clave,gasto,gastos,descripción
0,100013601,C002,30,C002,Jabón de barra
1,100013601,C004,84,C004,Suavizantes de telas
2,100013601,C005,60,C005,Limpiadores (en polvo o líquido)
3,100013601,D001,33,D001,Jabón de tocador
4,100013601,F007,2400,F007,Gasolina Magna


In [38]:
gastos_hogarprincipal['gasto'].describe()

count    759690.000000
mean        471.416237
std        3944.792201
min           0.000000
25%          35.000000
50%          80.000000
75%         240.000000
max      980000.000000
Name: gasto, dtype: float64

In [39]:
#ya no me interesa tener la clave de gasto
gastos_hogarprincipal=gastos_hogarprincipal[['folioviv','gasto', 'descripción']]

2.4 necesito una tabla con gasto como variable

In [40]:
#quiero que los gastos sean columna no fila
result_hogar = gastos_hogarprincipal.pivot_table('gasto', ['folioviv'], 'descripción')

In [41]:
result_hogar.head()

descripción,"Accesorios de hule y plástico: jabonera, tapetes, espejos, etcétera","Accesorios para computadora (mouse, memorias usb, etcétera)","Accesorios: bocinas, audífonos, antena aérea, control remoto, etcétera","Accesorios: espejos, manijas, antenas, etcétera","Aceites, lubricantes y aditivos",Acumulador,"Agujas, cierres, botones y broches, tabla para tejer, etcétera","Alcohol, merthiolate, solución antiséptica, botiquín de primeros auxilios, etcétera",Alfombras y tapetes,"Algodón, gasas, vendas, jeringas, etcétera","Alquiler de televisión, videocaseteras, computadoras, etcétera",Alquiler de terrenos,Anteojos y lentes de contacto e intraoculares,"Análisis clínicos y estudios médicos (rayos X, electros, etcétera )","Análisis clínicos y estudios médicos: Rayos X, ultrasonido, etcétera, durante el parto",Análisis clínicos y estudios médicos: rayos X,"Análisis clínicos y estudios médicos: rayos X, ultrasonidos, durante el embarazo",Aparatos de aire acondicionado para casa (incluye refrigeración o clima),"Aparatos ortopédicos y para terapia, silla de ruedas, andadera, muletas, etcétera",Aparatos para sordera,Aparatos telefónicos,"Artículos de deporte y cacería (aparatos para ejercicio, etcétera), balón de fútbol, alberca inflable","Artículos de jardinería: plantas, flores, macetas, tierra, abono, arreglos florales naturales, fertilizantes","Artículos de tocador para bebé, toallas húmedas, talco para bebé","Artículos eléctricos: rasuradora, secadora, lámpara para uñas, plancha de cabello, etcétera","Artículos y accesorios para el cuidado del bebé, bañera, carriola, pañalera, cobertor, moisés, chupón, esterilizador de biberones",Aspiradora,"Audiocasetes, discos y discos compactos, Itunes, películas en DVD",Automóvil y/o guayín,"Ayuda a parientes y personas ajenas al hogar, pago de renta a otro hogar",Batería de cocina y piezas sueltas,Batidora,Baños y masajes,Bicicleta,Blanqueadores,Blusas para mujer de 18 o más años,Blusas para niña de 0 a 4 años,Blusas para niña de 5 a 17 años,Bolsas,Botas para hombre de 18 o más años,...,Tenis para mujer de 18 o más años,Tenis para niña de 0 a 4 años,Tenis para niña de 5 a 17 años,Tenis para niño de 0 a 4 años,Tenis para niño de 5 a 17 años,Tintes y líquidos para permanente,Tintorería,Toallas,Toallas sanitarias,"Trajes, sacos, abrigos, gabardinas e impermeables para niño de 5 a 17 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para hombre de 18 o más años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para mujer de 18 o más años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niña de 0 a 4 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niña de 5 a 17 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niño de 0 a 4 años",Transporte aéreo,Transporte escolar,Transporte ferroviario,Transporte foráneo,Tratamiento para el control de peso,Trusas y bóxer para hombre de 18 o más años,Trusas y bóxer para niño de 0 a 4 años,Trusas y bóxer para niño de 5 a 17 años,"Trámites para vehículos: licencias, placas, verificación vehicular, etcétera","Uniformes y prendas de vestir para actividades educativas, artísticas y deportivas","Vajilla completa de cristal, barro, plástico, etcétera","Vasos, copas y jarras de cristal, plástico, cerámica, etcétera",Velas y veladoras,Ventilador,Vestidos para mujer de 18 o más años,Vestidos para niña de 5 a 17 años,Vestidos y faldas para niña de 0 a 4 años,"Videocasetes, cartuchos y discos para videojuegos, tarjetas para descargas de videojuegos","Vitaminas y complementos alimenticios, durante el embarazo",Zapatos para hombre de 18 o más años,Zapatos para niño de 0 a 4 años,Zapatos para niño de 5 a 17 años,Zapatos y sandalias para vestir para mujer de 18 o más años,Zapatos y sandalias para vestir para niña de 0 a 4 años,Zapatos y sandalias para vestir para niña de 5 a 17 años
folioviv,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000214

In [42]:
# que significan los NaN?
result_hogar=result_hogar.fillna(0)

In [43]:
result_hogar.head()

descripción,"Accesorios de hule y plástico: jabonera, tapetes, espejos, etcétera","Accesorios para computadora (mouse, memorias usb, etcétera)","Accesorios: bocinas, audífonos, antena aérea, control remoto, etcétera","Accesorios: espejos, manijas, antenas, etcétera","Aceites, lubricantes y aditivos",Acumulador,"Agujas, cierres, botones y broches, tabla para tejer, etcétera","Alcohol, merthiolate, solución antiséptica, botiquín de primeros auxilios, etcétera",Alfombras y tapetes,"Algodón, gasas, vendas, jeringas, etcétera","Alquiler de televisión, videocaseteras, computadoras, etcétera",Alquiler de terrenos,Anteojos y lentes de contacto e intraoculares,"Análisis clínicos y estudios médicos (rayos X, electros, etcétera )","Análisis clínicos y estudios médicos: Rayos X, ultrasonido, etcétera, durante el parto",Análisis clínicos y estudios médicos: rayos X,"Análisis clínicos y estudios médicos: rayos X, ultrasonidos, durante el embarazo",Aparatos de aire acondicionado para casa (incluye refrigeración o clima),"Aparatos ortopédicos y para terapia, silla de ruedas, andadera, muletas, etcétera",Aparatos para sordera,Aparatos telefónicos,"Artículos de deporte y cacería (aparatos para ejercicio, etcétera), balón de fútbol, alberca inflable","Artículos de jardinería: plantas, flores, macetas, tierra, abono, arreglos florales naturales, fertilizantes","Artículos de tocador para bebé, toallas húmedas, talco para bebé","Artículos eléctricos: rasuradora, secadora, lámpara para uñas, plancha de cabello, etcétera","Artículos y accesorios para el cuidado del bebé, bañera, carriola, pañalera, cobertor, moisés, chupón, esterilizador de biberones",Aspiradora,"Audiocasetes, discos y discos compactos, Itunes, películas en DVD",Automóvil y/o guayín,"Ayuda a parientes y personas ajenas al hogar, pago de renta a otro hogar",Batería de cocina y piezas sueltas,Batidora,Baños y masajes,Bicicleta,Blanqueadores,Blusas para mujer de 18 o más años,Blusas para niña de 0 a 4 años,Blusas para niña de 5 a 17 años,Bolsas,Botas para hombre de 18 o más años,...,Tenis para mujer de 18 o más años,Tenis para niña de 0 a 4 años,Tenis para niña de 5 a 17 años,Tenis para niño de 0 a 4 años,Tenis para niño de 5 a 17 años,Tintes y líquidos para permanente,Tintorería,Toallas,Toallas sanitarias,"Trajes, sacos, abrigos, gabardinas e impermeables para niño de 5 a 17 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para hombre de 18 o más años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para mujer de 18 o más años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niña de 0 a 4 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niña de 5 a 17 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niño de 0 a 4 años",Transporte aéreo,Transporte escolar,Transporte ferroviario,Transporte foráneo,Tratamiento para el control de peso,Trusas y bóxer para hombre de 18 o más años,Trusas y bóxer para niño de 0 a 4 años,Trusas y bóxer para niño de 5 a 17 años,"Trámites para vehículos: licencias, placas, verificación vehicular, etcétera","Uniformes y prendas de vestir para actividades educativas, artísticas y deportivas","Vajilla completa de cristal, barro, plástico, etcétera","Vasos, copas y jarras de cristal, plástico, cerámica, etcétera",Velas y veladoras,Ventilador,Vestidos para mujer de 18 o más años,Vestidos para niña de 5 a 17 años,Vestidos y faldas para niña de 0 a 4 años,"Videocasetes, cartuchos y discos para videojuegos, tarjetas para descargas de videojuegos","Vitaminas y complementos alimenticios, durante el embarazo",Zapatos para hombre de 18 o más años,Zapatos para niño de 0 a 4 años,Zapatos para niño de 5 a 17 años,Zapatos y sandalias para vestir para mujer de 18 o más años,Zapatos y sandalias para vestir para niña de 0 a 4 años,Zapatos y sandalias para vestir para niña de 5 a 17 años
folioviv,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000214

In [44]:
#verifico cuantos hogares tengo para ver que todo siga bien
gastos_hogarprincipal['folioviv'].nunique()

32380

In [45]:
#necesito conservar el folioviv como variable no como index
result_hogar=result_hogar.reset_index()

In [46]:
ingreso_todos['folioviv']=ingreso_todos['folioviv'].astype(str)

2.5 unimos los ingresos

In [47]:
gastos_ingresos_hogar= pd.merge(result_hogar, ingreso_todos, on='folioviv')

In [48]:
gastos_ingresos_hogar['folioviv'].nunique()

32377

In [49]:
gastos_ingresos_hogar.head()

,folioviv,"Accesorios de hule y plástico: jabonera, tapetes, espejos, etcétera","Accesorios para computadora (mouse, memorias usb, etcétera)","Accesorios: bocinas, audífonos, antena aérea, control remoto, etcétera","Accesorios: espejos, manijas, antenas, etcétera","Aceites, lubricantes y aditivos",Acumulador,"Agujas, cierres, botones y broches, tabla para tejer, etcétera","Alcohol, merthiolate, solución antiséptica, botiquín de primeros auxilios, etcétera",Alfombras y tapetes,"Algodón, gasas, vendas, jeringas, etcétera","Alquiler de televisión, videocaseteras, computadoras, etcétera",Alquiler de terrenos,Anteojos y lentes de contacto e intraoculares,"Análisis clínicos y estudios médicos (rayos X, electros, etcétera )","Análisis clínicos y estudios médicos: Rayos X, ultrasonido, etcétera, durante el parto",Análisis clínicos y estudios médicos: rayos X,"Análisis clínicos y estudios médicos: rayos X, ultrasonidos, durante el embarazo",Aparatos de aire acondicionado para casa (incluye refrigeración o clima),"Aparatos ortopédicos y para terapia, silla de ruedas, andadera, muletas, etcétera",Aparatos para sordera,Aparatos telefónicos,"Artículos de deporte y cacería (aparatos para ejercicio, etcétera), balón de fútbol, alberca inflable","Artículos de jardinería: plantas, flores, macetas, tierra, abono, arreglos florales naturales, fertilizantes","Artículos de tocador para bebé, toallas húmedas, talco para bebé","Artículos eléctricos: rasuradora, secadora, lámpara para uñas, plancha de cabello, etcétera","Artículos y accesorios para el cuidado del bebé, bañera, carriola, pañalera, cobertor, moisés, chupón, esterilizador de biberones",Aspiradora,"Audiocasetes, discos y discos compactos, Itunes, películas en DVD",Automóvil y/o guayín,"Ayuda a parientes y personas ajenas al hogar, pago de renta a otro hogar",Batería de cocina y piezas sueltas,Batidora,Baños y masajes,Bicicleta,Blanqueadores,Blusas para mujer de 18 o más años,Blusas para niña de 0 a 4 años,Blusas para niña de 5 a 17 años,Bolsas,...,Toallas,Toallas sanitarias,"Trajes, sacos, abrigos, gabardinas e impermeables para niño de 5 a 17 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para hombre de 18 o más años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para mujer de 18 o más años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niña de 0 a 4 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niña de 5 a 17 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niño de 0 a 4 años",Transporte aéreo,Transporte escolar,Transporte ferroviario,Transporte foráneo,Tratamiento para el control de peso,Trusas y bóxer para hombre de 18 o más años,Trusas y bóxer para niño de 0 a 4 años,Trusas y bóxer para niño de 5 a 17 años,"Trámites para vehículos: licencias, placas, verificación vehicular, etcétera","Uniformes y prendas de vestir para actividades educativas, artísticas y deportivas","Vajilla completa de cristal, barro, plástico, etcétera","Vasos, copas y jarras de cristal, plástico, cerámica, etcétera",Velas y veladoras,Ventilador,Vestidos para mujer de 18 o más años,Vestidos para niña de 5 a 17 años,Vestidos y faldas para niña de 0 a 4 años,"Videocasetes, cartuchos y discos para videojuegos, tarjetas para descargas de videojuegos","Vitaminas y complementos alimenticios, durante el embarazo",Zapatos para hombre de 18 o más años,Zapatos para niño de 0 a 4 años,Zapatos para niño de 5 a 17 años,Zapatos y sandalias para vestir para mujer de 18 o más años,Zapatos y sandalias para vestir para niña de 0 a 4 años,Zapatos y sandalias para vestir para niña de 5 a 17 años,ing_1,ing_2,ing_3,ing_4,ing_5,ing_6,mean_income
0,1000021401,0.0,0.0,0.0,0.0,0.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,...,0.0,30.0,0.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,0.0,0.0,0.0,

In [50]:
gastos_ingresos_hogar.drop(['ing_1', 'ing_2', 'ing_3', 'ing_4', 'ing_5', 'ing_6'], axis=1)

,folioviv,"Accesorios de hule y plástico: jabonera, tapetes, espejos, etcétera","Accesorios para computadora (mouse, memorias usb, etcétera)","Accesorios: bocinas, audífonos, antena aérea, control remoto, etcétera","Accesorios: espejos, manijas, antenas, etcétera","Aceites, lubricantes y aditivos",Acumulador,"Agujas, cierres, botones y broches, tabla para tejer, etcétera","Alcohol, merthiolate, solución antiséptica, botiquín de primeros auxilios, etcétera",Alfombras y tapetes,"Algodón, gasas, vendas, jeringas, etcétera","Alquiler de televisión, videocaseteras, computadoras, etcétera",Alquiler de terrenos,Anteojos y lentes de contacto e intraoculares,"Análisis clínicos y estudios médicos (rayos X, electros, etcétera )","Análisis clínicos y estudios médicos: Rayos X, ultrasonido, etcétera, durante el parto",Análisis clínicos y estudios médicos: rayos X,"Análisis clínicos y estudios médicos: rayos X, ultrasonidos, durante el embarazo",Aparatos de aire acondicionado para casa (incluye refrigeración o clima),"Aparatos ortopédicos y para terapia, silla de ruedas, andadera, muletas, etcétera",Aparatos para sordera,Aparatos telefónicos,"Artículos de deporte y cacería (aparatos para ejercicio, etcétera), balón de fútbol, alberca inflable","Artículos de jardinería: plantas, flores, macetas, tierra, abono, arreglos florales naturales, fertilizantes","Artículos de tocador para bebé, toallas húmedas, talco para bebé","Artículos eléctricos: rasuradora, secadora, lámpara para uñas, plancha de cabello, etcétera","Artículos y accesorios para el cuidado del bebé, bañera, carriola, pañalera, cobertor, moisés, chupón, esterilizador de biberones",Aspiradora,"Audiocasetes, discos y discos compactos, Itunes, películas en DVD",Automóvil y/o guayín,"Ayuda a parientes y personas ajenas al hogar, pago de renta a otro hogar",Batería de cocina y piezas sueltas,Batidora,Baños y masajes,Bicicleta,Blanqueadores,Blusas para mujer de 18 o más años,Blusas para niña de 0 a 4 años,Blusas para niña de 5 a 17 años,Bolsas,...,Tenis para niña de 0 a 4 años,Tenis para niña de 5 a 17 años,Tenis para niño de 0 a 4 años,Tenis para niño de 5 a 17 años,Tintes y líquidos para permanente,Tintorería,Toallas,Toallas sanitarias,"Trajes, sacos, abrigos, gabardinas e impermeables para niño de 5 a 17 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para hombre de 18 o más años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para mujer de 18 o más años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niña de 0 a 4 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niña de 5 a 17 años","Trajes, sacos, conjuntos, abrigos, gabardinas e impermeables para niño de 0 a 4 años",Transporte aéreo,Transporte escolar,Transporte ferroviario,Transporte foráneo,Tratamiento para el control de peso,Trusas y bóxer para hombre de 18 o más años,Trusas y bóxer para niño de 0 a 4 años,Trusas y bóxer para niño de 5 a 17 años,"Trámites para vehículos: licencias, placas, verificación vehicular, etcétera","Uniformes y prendas de vestir para actividades educativas, artísticas y deportivas","Vajilla completa de cristal, barro, plástico, etcétera","Vasos, copas y jarras de cristal, plástico, cerámica, etcétera",Velas y veladoras,Ventilador,Vestidos para mujer de 18 o más años,Vestidos para niña de 5 a 17 años,Vestidos y faldas para niña de 0 a 4 años,"Videocasetes, cartuchos y discos para videojuegos, tarjetas para descargas de videojuegos","Vitaminas y complementos alimenticios, durante el embarazo",Zapatos para hombre de 18 o más años,Zapatos para niño de 0 a 4 años,Zapatos para niño de 5 a 17 años,Zapatos y sandalias para vestir para mujer de 18 o más años,Zapatos y sandalias para vestir para niña de 0 a 4 años,Zapatos y sandalias para vestir para niña de 5 a 17 años,mean_income
0,1000021401,0.0,0.0,0.0,0.0,0.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
#quiero datos de la vivienda para hacer perfiles demograficos

# 3.0 datos vivienda

In [52]:
datos_vivienda = pd.read_csv(r'conjunto_de_datos_viviendas_enigh_2018_ns.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
datos_vivienda['folioviv'].dtypes

dtype('int64')

In [54]:
datos_vivienda['folioviv']=datos_vivienda['folioviv'].astype(str)

In [55]:
gastos_ingresos_vivienda= pd.merge(gastos_ingresos_hogar, datos_vivienda, on='folioviv', how='left')

In [56]:
gastos_ingresos_vivienda['folioviv'].nunique()

32377

In [57]:
#tengo la columna ubicageo pero quisiera tenerlos por nombre 

# 4.0 catalogo de municipios

In [58]:
ubica_geo=pd.read_csv(r'ubic_geo.csv', encoding='latin-1')

In [59]:
ubica_geo.head()

,ubic_geo,entidad,desc_ent,municipio,des_mun
0,0,0,No aplica,0,No aplica
1,1000,1,Aguascalientes,1000,No identificado
2,1001,1,Aguascalientes,1001,Aguascalientes
3,1002,1,Aguascalientes,1002,Asientos
4,1003,1,Aguascalientes,1003,Calvillo


In [60]:
gastos_ingresos_vivienda= pd.merge(gastos_ingresos_vivienda, ubica_geo, left_on='ubica_geo', right_on='ubic_geo',how='left')

In [61]:
gastos_ingresos_vivienda.head()

,folioviv,"Accesorios de hule y plástico: jabonera, tapetes, espejos, etcétera","Accesorios para computadora (mouse, memorias usb, etcétera)","Accesorios: bocinas, audífonos, antena aérea, control remoto, etcétera","Accesorios: espejos, manijas, antenas, etcétera","Aceites, lubricantes y aditivos",Acumulador,"Agujas, cierres, botones y broches, tabla para tejer, etcétera","Alcohol, merthiolate, solución antiséptica, botiquín de primeros auxilios, etcétera",Alfombras y tapetes,"Algodón, gasas, vendas, jeringas, etcétera","Alquiler de televisión, videocaseteras, computadoras, etcétera",Alquiler de terrenos,Anteojos y lentes de contacto e intraoculares,"Análisis clínicos y estudios médicos (rayos X, electros, etcétera )","Análisis clínicos y estudios médicos: Rayos X, ultrasonido, etcétera, durante el parto",Análisis clínicos y estudios médicos: rayos X,"Análisis clínicos y estudios médicos: rayos X, ultrasonidos, durante el embarazo",Aparatos de aire acondicionado para casa (incluye refrigeración o clima),"Aparatos ortopédicos y para terapia, silla de ruedas, andadera, muletas, etcétera",Aparatos para sordera,Aparatos telefónicos,"Artículos de deporte y cacería (aparatos para ejercicio, etcétera), balón de fútbol, alberca inflable","Artículos de jardinería: plantas, flores, macetas, tierra, abono, arreglos florales naturales, fertilizantes","Artículos de tocador para bebé, toallas húmedas, talco para bebé","Artículos eléctricos: rasuradora, secadora, lámpara para uñas, plancha de cabello, etcétera","Artículos y accesorios para el cuidado del bebé, bañera, carriola, pañalera, cobertor, moisés, chupón, esterilizador de biberones",Aspiradora,"Audiocasetes, discos y discos compactos, Itunes, películas en DVD",Automóvil y/o guayín,"Ayuda a parientes y personas ajenas al hogar, pago de renta a otro hogar",Batería de cocina y piezas sueltas,Batidora,Baños y masajes,Bicicleta,Blanqueadores,Blusas para mujer de 18 o más años,Blusas para niña de 0 a 4 años,Blusas para niña de 5 a 17 años,Bolsas,...,estim_pago,pago_viv,pago_mesp,tipo_adqui,viv_usada,tipo_finan,num_dueno1,hog_dueno1,num_dueno2,hog_dueno2,escrituras,lavadero,fregadero,regadera,tinaco_azo,cisterna,pileta,calent_sol,calent_gas,medidor_luz,bomba_agua,tanque_gas,aire_acond,calefacc,tot_resid,tot_hom,tot_muj,tot_hog,ubica_geo,tam_loc,est_socio,est_dis,upm,factor,procaptar,ubic_geo,entidad,desc_ent,municipio,des_mun
0,1000021401,0.0,0.0,0.0,0.0,0.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,...,8000,,,1,1,5,1,1,,,1,1,1,1,1,1,2,2,1,1,1,2,2,2,3,2,1,1,10005,1,4,146,2877,332,0,10005,10,Durango,10005,Durango
1,1000021402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,...,,,1,,,,,,,,,1,1,1,1,2,1,2,1,1,1,2,2,2,5,3,2,1,10005,1,4,146,2877,332,0,10005,10,Durango,10005,Durango
2,1000021404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,...,,,1,,,,,,,,,1,1,1,1,2,2,2,1,1,1,2,2,2,1,1,0,1,10005,1,4,146,2877,332,0,10005,10,Durango,10005,Durango
3,1000021405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,0.0,0.0,0.0,0.0,...,,,1,,,,,,,,,1,1,1,1,1,2,2,1,1,1,2,2,2,4,3,1,1,10005,1,4,146,2877,332,0,10005,10,Durango,10005,Durango
4,1000035901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.0,370.0,0.0,0.0,0.0,...,4500,,,1,2,2,1,1,,,1,1,1,1,1,1,1,2,1,1,1,2,2,2,3,2,1,1,10005,1,3,145,2878,270,0,10005,10,Durango,10005,Durango


In [64]:
gastos_ingresos_vivienda.sample(n=20)

,folioviv,"Accesorios de hule y plástico: jabonera, tapetes, espejos, etcétera","Accesorios para computadora (mouse, memorias usb, etcétera)","Accesorios: bocinas, audífonos, antena aérea, control remoto, etcétera","Accesorios: espejos, manijas, antenas, etcétera","Aceites, lubricantes y aditivos",Acumulador,"Agujas, cierres, botones y broches, tabla para tejer, etcétera","Alcohol, merthiolate, solución antiséptica, botiquín de primeros auxilios, etcétera",Alfombras y tapetes,"Algodón, gasas, vendas, jeringas, etcétera","Alquiler de televisión, videocaseteras, computadoras, etcétera",Alquiler de terrenos,Anteojos y lentes de contacto e intraoculares,"Análisis clínicos y estudios médicos (rayos X, electros, etcétera )","Análisis clínicos y estudios médicos: Rayos X, ultrasonido, etcétera, durante el parto",Análisis clínicos y estudios médicos: rayos X,"Análisis clínicos y estudios médicos: rayos X, ultrasonidos, durante el embarazo",Aparatos de aire acondicionado para casa (incluye refrigeración o clima),"Aparatos ortopédicos y para terapia, silla de ruedas, andadera, muletas, etcétera",Aparatos para sordera,Aparatos telefónicos,"Artículos de deporte y cacería (aparatos para ejercicio, etcétera), balón de fútbol, alberca inflable","Artículos de jardinería: plantas, flores, macetas, tierra, abono, arreglos florales naturales, fertilizantes","Artículos de tocador para bebé, toallas húmedas, talco para bebé","Artículos eléctricos: rasuradora, secadora, lámpara para uñas, plancha de cabello, etcétera","Artículos y accesorios para el cuidado del bebé, bañera, carriola, pañalera, cobertor, moisés, chupón, esterilizador de biberones",Aspiradora,"Audiocasetes, discos y discos compactos, Itunes, películas en DVD",Automóvil y/o guayín,"Ayuda a parientes y personas ajenas al hogar, pago de renta a otro hogar",Batería de cocina y piezas sueltas,Batidora,Baños y masajes,Bicicleta,Blanqueadores,Blusas para mujer de 18 o más años,Blusas para niña de 0 a 4 años,Blusas para niña de 5 a 17 años,Bolsas,...,estim_pago,pago_viv,pago_mesp,tipo_adqui,viv_usada,tipo_finan,num_dueno1,hog_dueno1,num_dueno2,hog_dueno2,escrituras,lavadero,fregadero,regadera,tinaco_azo,cisterna,pileta,calent_sol,calent_gas,medidor_luz,bomba_agua,tanque_gas,aire_acond,calefacc,tot_resid,tot_hom,tot_muj,tot_hog,ubica_geo,tam_loc,est_socio,est_dis,upm,factor,procaptar,ubic_geo,entidad,desc_ent,municipio,des_mun
7920,1261117401,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,...,600,,,3,,4,1,1,,,1,1,2,2,2,2,1,2,2,2,2,2,2,2,5,3,2,1,12048,4,1,195,3534,358,0,12048,12,Guerrero,12048,Petatlán
272,1000694706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130.0,0.0,0.0,0.0,0.0,...,4000,,,2,,5,2,1,,,1,1,1,1,1,1,2,1,1,1,1,1,2,2,3,2,1,1,10005,1,2,144,2931,215,0,10005,10,Durango,10005,Durango
24644,660102515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0,410.0,0.0,...,1000,,,3,,5,2,1,,,4,2,2,2,2,2,2,2,2,1,2,2,2,2,5,2,3,1,6009,4,1,94,1889,32,0,6009,6,Colima,6009,Tecomán
12747,207299804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,190.0,0.0,0.0,0.0,...,,,1,,,,,,,,,2,1,1,2,2,2,2,1,1,2,2,2,2,1,0,1,1,2004,2,2,14,536,613,0,2004,2,Baja California,2004,Tijuana
8671,1301602007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,6.0,0.0,0.0,0.0,0.0,...,,,1,,,,,,,,,1,2,1,1,2,1,2,1,1,1,2,2,2,4,3,1,1,13069,2,2,206,3612,407,0,13069,13,Hidalgo,13069,Tizayuca
12105,203847904,0.0,0.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,320.0,0.0,0.

In [65]:
df_final = gastos_ingresos_vivienda.replace(to_replace ='\s\s*', value = '0', regex = True) 

In [72]:
df_final.sample(n=20)

,folioviv,"Accesorios de hule y plástico: jabonera, tapetes, espejos, etcétera","Accesorios para computadora (mouse, memorias usb, etcétera)","Accesorios: bocinas, audífonos, antena aérea, control remoto, etcétera","Accesorios: espejos, manijas, antenas, etcétera","Aceites, lubricantes y aditivos",Acumulador,"Agujas, cierres, botones y broches, tabla para tejer, etcétera","Alcohol, merthiolate, solución antiséptica, botiquín de primeros auxilios, etcétera",Alfombras y tapetes,"Algodón, gasas, vendas, jeringas, etcétera","Alquiler de televisión, videocaseteras, computadoras, etcétera",Alquiler de terrenos,Anteojos y lentes de contacto e intraoculares,"Análisis clínicos y estudios médicos (rayos X, electros, etcétera )","Análisis clínicos y estudios médicos: Rayos X, ultrasonido, etcétera, durante el parto",Análisis clínicos y estudios médicos: rayos X,"Análisis clínicos y estudios médicos: rayos X, ultrasonidos, durante el embarazo",Aparatos de aire acondicionado para casa (incluye refrigeración o clima),"Aparatos ortopédicos y para terapia, silla de ruedas, andadera, muletas, etcétera",Aparatos para sordera,Aparatos telefónicos,"Artículos de deporte y cacería (aparatos para ejercicio, etcétera), balón de fútbol, alberca inflable","Artículos de jardinería: plantas, flores, macetas, tierra, abono, arreglos florales naturales, fertilizantes","Artículos de tocador para bebé, toallas húmedas, talco para bebé","Artículos eléctricos: rasuradora, secadora, lámpara para uñas, plancha de cabello, etcétera","Artículos y accesorios para el cuidado del bebé, bañera, carriola, pañalera, cobertor, moisés, chupón, esterilizador de biberones",Aspiradora,"Audiocasetes, discos y discos compactos, Itunes, películas en DVD",Automóvil y/o guayín,"Ayuda a parientes y personas ajenas al hogar, pago de renta a otro hogar",Batería de cocina y piezas sueltas,Batidora,Baños y masajes,Bicicleta,Blanqueadores,Blusas para mujer de 18 o más años,Blusas para niña de 0 a 4 años,Blusas para niña de 5 a 17 años,Bolsas,...,estim_pago,pago_viv,pago_mesp,tipo_adqui,viv_usada,tipo_finan,num_dueno1,hog_dueno1,num_dueno2,hog_dueno2,escrituras,lavadero,fregadero,regadera,tinaco_azo,cisterna,pileta,calent_sol,calent_gas,medidor_luz,bomba_agua,tanque_gas,aire_acond,calefacc,tot_resid,tot_hom,tot_muj,tot_hog,ubica_geo,tam_loc,est_socio,est_dis,upm,factor,procaptar,ubic_geo,entidad,desc_ent,municipio,des_mun
30172,860919006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0,0,1,2,1,2,2,2,2,2,1,2,2,2,2,1,0,1,1,8047,4,1,135,2522,139,0,8047,8,Chihuahua,8047,Moris
13374,260063519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,...,500,0,0,4,0,0,1,1,0,0,3,2,2,2,2,2,2,2,2,1,2,2,2,2,1,1,0,1,2002,4,2,28,574,50,0,2002,2,Baja0California,2002,Mexicali
29311,860257711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,1000,0,0,1,1,5,1,1,0,0,1,2,1,1,1,2,2,1,2,1,1,2,2,2,2,1,1,1,8012,4,2,136,2478,109,0,8012,8,Chihuahua,8012,Carichí
17202,400142605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0,0,2,2,1,1,2,2,2,2,1,2,2,1,2,4,2,2,1,4002,1,2,42,869,143,0,4002,4,Campeche,4002,Campeche
15136,300410202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,...,5000,0,0,2,0,5,2,1,0,0,1,1,1,1,1,2,1,2,1,1,2,2,1,2,2,1,1,1,3003,1,4,31,718,126,0,3003,3,Baja0California0Sur,3003,La0Paz
28625,808387416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [70]:
df_final.info(verbose =True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32377 entries, 0 to 32376
Data columns (total 525 columns):
 #   Column                                                                                                                                                                                               Dtype  
---  ------                                                                                                                                                                                               -----  
 0   folioviv                                                                                                                                                                                             object 
 1   Accesorios de hule y plástico: jabonera, tapetes, espejos, etcétera                                                                                                                                  float64
 2   Accesorios para computadora (mouse, memorias 

In [71]:
# esta completa nuestra base la guardamos:
df_final.to_csv(r'gastos_ingresos_vivienda_geografico_enigh18_final.csv')

In [ ]:
/content/gastos_ingresos_vivienda_geografico_enigh18.csv
/content/gastos_ingresos_vivienda_geografico_enigh18_final.csv